# Objective

+ Build a sentiment analysis model with huggingface (HF) models (Includes pre-trained & fine-tuning)
+ Using HF api's instead of native pytorch


In [1]:
# !pip install transformers
# !pip install torch

In [1]:
import transformers
import gc
import torch
# import mlflow.pytorch
import random
import numpy as np
import pandas as pd
# import seaborn as sns
# from pylab import rcParams
import matplotlib.pyplot as plt
import torch.nn.functional as F

from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import (BertModel, \
BertTokenizer, \
BertConfig,\
AdamW, \
get_linear_schedule_with_warmup, \
pipeline, \
BertForSequenceClassification, \
Trainer, \
TrainingArguments
)

get_ipython().run_line_magic('matplotlib', 'inline')

In [2]:
# Checking for cuda on the machine
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# Use a pre-trained HF pipeline for sentiment analysis
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

In [4]:
classifier('I’ve been signed up for a year and haven’t received a single sample for anything. \
I’d never complain about “free” unless “free” meant nothing at all. lol')

[{'label': 'NEGATIVE', 'score': 0.9647619724273682}]

In [5]:
# Set the seeds for reproducibility of results
SEED = 2021

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [6]:
df_org = pd.read_csv("./complaints_small.csv", nrows=5000)
print(df_org.shape)
df_org.head()

(5000, 18)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,13-06-2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,CAPITAL ONE FINANCIAL CORPORATION,PA,186XX,NaN,Consent not provided,Web,13-06-2019,Closed with explanation,Yes,NaN,3274605
1,01-11-2019,Vehicle loan or lease,Loan,Struggling to pay your loan,Denied request to lower payments,I contacted Ally on Friday XX/XX/XXXX after fa...,Company has responded to the consumer and the ...,ALLY FINANCIAL INC.,NJ,088XX,NaN,Consent provided,Web,01-11-2019,Closed with explanation,Yes,NaN,3425257
2,01-04-2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",PA,19067,NaN,Consent not provided,Web,01-04-2019,Closed with explanation,Yes,NaN,3198225
3,06-10-2021,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,"EQUIFAX, INC.",MD,21207,NaN,NaN,Web,06-10-2021,Closed with explanation,Yes,NaN,4783820
4,08-08-2019,Mortgage,Conventional home mortgage,Trouble during payment process,NaN,NaN,Company has responded to the consumer and the ...,"FLAGSTAR BANK, FSB",ID,83706,NaN,NaN,Referral,15-08-2019,Closed with explanation,Yes,NaN,3342290


In [7]:
# Considering only 2 cols which we need for our modelling
df = df_org.loc[(df_org['Consumer complaint narrative'].notnull()), ['Consumer complaint narrative', 'Product']].copy().reset_index().drop('index', axis = 1)
df.columns = ['text', 'target']
print('Dataframe size - ', df.shape)
df.head()
        

Dataframe size -  (2593, 2)


,text,target
0,I contacted Ally on Friday XX/XX/XXXX after fa...,Vehicle loan or lease
1,Hello This complaint is against the three cred...,"Credit reporting, credit repair services, or o..."
2,I am a victim of Identity Theft & currently ha...,"Credit reporting, credit repair services, or o..."
3,Two accounts are still on my credit history af...,"Credit reporting, credit repair services, or o..."
4,Receiving daily telephone call ( s ) from XXXX...,"Credit reporting, credit repair services, or o..."


In [8]:
df.target.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    1118
Debt collection                                                                  474
Credit card or prepaid card                                                      285
Mortgage                                                                         242
Checking or savings account                                                      181
Student loan                                                                     116
Vehicle loan or lease                                                             63
Money transfer, virtual currency, or money service                                57
Payday loan, title loan, or personal loan                                         44
Credit reporting                                                                   4
Consumer Loan                                                                      3
Money transfers                                                  

In [9]:
unique_labels = list(df.target.value_counts().index)
unique_labels

['Credit reporting, credit repair services, or other personal consumer reports',
 'Debt collection',
 'Credit card or prepaid card',
 'Mortgage',
 'Checking or savings account',
 'Student loan',
 'Vehicle loan or lease',
 'Money transfer, virtual currency, or money service',
 'Payday loan, title loan, or personal loan',
 'Credit reporting',
 'Consumer Loan',
 'Money transfers',
 'Credit card',
 'Bank account or service']

In [10]:
# Create numeric labels as training cannot be done on string type
label_dict = {}
for i in range(len(unique_labels)):
    label_dict[unique_labels[i]] = i

label_dict

{'Credit reporting, credit repair services, or other personal consumer reports': 0,
 'Debt collection': 1,
 'Credit card or prepaid card': 2,
 'Mortgage': 3,
 'Checking or savings account': 4,
 'Student loan': 5,
 'Vehicle loan or lease': 6,
 'Money transfer, virtual currency, or money service': 7,
 'Payday loan, title loan, or personal loan': 8,
 'Credit reporting': 9,
 'Consumer Loan': 10,
 'Money transfers': 11,
 'Credit card': 12,
 'Bank account or service': 13}

In [11]:
# Replace the target variable value with numeric dictionary
df.target = df.target.replace(label_dict)
df.head()

,text,target
0,I contacted Ally on Friday XX/XX/XXXX after fa...,6
1,Hello This complaint is against the three cred...,0
2,I am a victim of Identity Theft & currently ha...,0
3,Two accounts are still on my credit history af...,0
4,Receiving daily telephone call ( s ) from XXXX...,0


In [12]:
X = list(df['text'])
y = list(df['target'])

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)


In [14]:
class_names = np.unique(df.target)
class_names

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
      dtype=int64)

In [15]:
N_CLASSES = len(class_names) # No. of classes of target variable
MAX_LEN = 160 # Max length for the text to be allowed

TEST_SIZE = 0.1 # Test data proportion
VAL_SIZE = 0.5 # Validation data proportion

BATCH_SIZE = 8 # Batch size for each iteration
DROPOUT = 0.3 # Dropout for introducing regularization
EPOCHS = 1 # No. of epochs

# BERT modules required for configurtion, Classification & Tokenization
MODEL_CLASSES = {'bert': (BertConfig, BertForSequenceClassification, BertTokenizer, 'bert-base-cased')}

SELECTED_MODEL_CLASS = 'bert'

In [16]:
# Loading the BERT modules into objects
config_class, model_class, tokenizer_class, pretrained_model = MODEL_CLASSES[SELECTED_MODEL_CLASS]

In [17]:
# The tokenizer is a BERT tokenizer which converts the text to tokens
tokenizer = tokenizer_class.from_pretrained(pretrained_model, return_dict=False)

In [18]:
model = model_class.from_pretrained(\
pretrained_model, # Use the 12-layer BERT model, with an cased vocab.\
num_labels = N_CLASSES, # The number of output labels--2 for binary classification.\
output_attentions = False, # Whether the model returns attentions weights.\
output_hidden_states = False, # Whether the model returns all hidden-states.\
)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [19]:
# Tell pytorch to run this model on the GPU.
# model.cuda()

In [20]:
# Apply the tokenizer on the train and val data alongwith padding, truncation and max length
train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=MAX_LEN)
val_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=MAX_LEN)

In [21]:
len(val_encodings['input_ids'][0])

160

In [22]:
# Convert the dataset into tensors for modelling
class PrepareDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [23]:
# Convert our tokenized data into a torch Dataset
train_dataset = PrepareDataset(train_encodings, y_train)
val_dataset = PrepareDataset(val_encodings, y_val)

In [24]:
train_dataset

In [56]:
# Compute model metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1) # Gets the rowwise maximum value position
    # calculate accuracy using sklearn's function
    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    recall = recall_score(y_true=labels, y_pred=preds, average='macro')
    precision = precision_score(y_true=labels, y_pred=preds, average='macro')
    f1 = f1_score(y_true=labels, y_pred=preds, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 


In [57]:
# Training arguments
training_args = TrainingArguments( \
	output_dir='./results', \
	num_train_epochs=EPOCHS,\
	per_device_train_batch_size=BATCH_SIZE, \
	per_device_eval_batch_size=BATCH_SIZE, \
	warmup_steps=100, \
	weight_decay=0.01, \
	logging_dir='./logs', \
	load_best_model_at_end=True, \
	evaluation_strategy="steps", \
	logging_steps=100, \
	save_steps=100 \
)
# warmup_steps: Number of steps used for a linear warmup from 0 to learning_rate
# evaluation_strategy: Number of update steps between two evaluations

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
# Create a trainer object
trainer = Trainer(
model=model, \
args=training_args, \
train_dataset=train_dataset, \
eval_dataset=val_dataset,\
compute_metrics=compute_metrics, \
)


In [59]:
# Clear grabage/cache
gc.collect()
torch.cuda.empty_cache()

In [60]:
# Trigger the model training
# mlflow.end_run()
trainer.train()

# The loss function here is CrossEntropyLoss = -summation(ti * log(pi)) where ti is the binary truth label and pi is the softmax probability 

***** Running training *****
  Num examples = 2074
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 260


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,2.038800,1.424462,0.589595,0.297917,0.279778,0.246077
200,1.204900,1.030066,0.693642,0.346009,0.371583,0.355845


***** Running Evaluation *****
  Num examples = 519
  Batch size = 8
C:\Users\Karthik\Programs\Anaconda3\envs\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./results\checkpoint-100
Configuration saved in ./results\checkpoint-100\config.json
Model weights saved in ./results\checkpoint-100\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 519
  Batch size = 8
C:\Users\Karthik\Programs\Anaconda3\envs\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./results\checkpoint-

TrainOutput(global_step=260, training_loss=1.4783022953913763, metrics={'train_runtime': 3427.7384, 'train_samples_per_second': 0.605, 'train_steps_per_second': 0.076, 'total_flos': 170547226072320.0, 'train_loss': 1.4783022953913763, 'epoch': 1.0})

In [61]:
# Evaluate the model
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 519
  Batch size = 8


C:\Users\Karthik\Programs\Anaconda3\envs\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 1.030065655708313,
 'eval_accuracy': 0.6936416184971098,
 'eval_precision': 0.3460093604668385,
 'eval_recall': 0.371582907127797,
 'eval_f1': 0.3558451522345688,
 'eval_runtime': 223.1035,
 'eval_samples_per_second': 2.326,
 'eval_steps_per_second': 0.291,
 'epoch': 1.0}

## Save model

In [63]:
# Saving the fine tuned model & tokenizer
model_path = "sentiment-"+pretrained_model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


Configuration saved in sentiment-bert-base-cased\config.json
Model weights saved in sentiment-bert-base-cased\pytorch_model.bin
tokenizer config file saved in sentiment-bert-base-cased\tokenizer_config.json
Special tokens file saved in sentiment-bert-base-cased\special_tokens_map.json


('sentiment-bert-base-cased\\tokenizer_config.json',
 'sentiment-bert-base-cased\\special_tokens_map.json',
 'sentiment-bert-base-cased\\vocab.txt',
 'sentiment-bert-base-cased\\added_tokens.json')

## Evaluation

In [64]:
# Load the saved model
model_load = model_class.from_pretrained(model_path, num_labels=N_CLASSES)

loading configuration file sentiment-bert-base-cased\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "

## Predicting on Raw Text

In [65]:
# Set the configs used for training
config = {
'MAX_LEN':MAX_LEN,
'TARGETS':class_names
}

In [67]:
def get_prediction(text, model, config):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=config['MAX_LEN'], return_tensors="pt")
    tokenizer(X_train, padding=True, truncation=True, max_length=MAX_LEN)
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return config['TARGETS'][probs.argmax()], probs



In [68]:
# New text for prediction
review_text = "I love completing my todos! Best app ever!!!"

In [69]:
# Predict
get_prediction(review_text, model_load, config)

# The output is class 0

(0,
 tensor([[0.4695, 0.1020, 0.1665, 0.0459, 0.0616, 0.0516, 0.0276, 0.0164, 0.0144,
          0.0071, 0.0168, 0.0064, 0.0079, 0.0061]], grad_fn=<SoftmaxBackward>))